In [ ]:
!pip install fastai

In [ ]:
import pandas as pd
from fastai.imports import *

In [ ]:
dataset_path = "/kaggle/input/us-patent-phrase-to-phrase-matching"
path.ls()

##   EDA

In [ ]:
if iskaggle: path = Path('../input/us-patent-phrase-to-phrase-matching')


In [ ]:
df_train = pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv')
df_train.info()
df_train.describe()
df_train

In [ ]:
eval_df = pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv')
eval_df.describe()
eval_df.info()
eval_df

In [ ]:
print(df_train.anchor.value_counts())
print(df_train.target.value_counts())
print(df_train.context.value_counts())

In [ ]:
#creating a column to separate the section from context
df_train["section"] = df_train.context.str[0]
df_train.section.value_counts()

In [ ]:
df_train[df_train.score==1]

In [ ]:
df.context.value_counts()

In [ ]:
df['section'] = df.context.str[0]
df.section.value_counts()

#### Histogram ANalysis

In [ ]:
df_train.score.hist()

## Training

#### Import libraries for training

In [ ]:
!pip install transformers
from torch.utils.data import DataLoader #load data using pytorch framework
import warnings,transformers,logging,torch #importing huggingface transformer
from transformers import TrainingArguments,Trainer #importing classes from huggingface to train and hadnle 
from transformers import AutoModelForSequenceClassification,AutoTokenizer #importing prtreained model

In [ ]:
warnings.simplefilter('ignore')
logging.disable(logging.WARNING)

In [ ]:
model_nm = 'microsoft/deberta-v3-small'

In [ ]:
model_nm = 'microsoft/deberta-v3-small'

### create a tokenizer for the model

In [ ]:
token = AutoTokenizer.from_pretrained(model_nm)

In [ ]:
sep = token.sep_token
df_train['input'] =  df_train.context + sep+ df_train.target + sep+ df_train.anchor
df_train.input.tail

#### convert pandas DataFrames into HuggingFace Datasets


In [ ]:
from datasets import Dataset, DatasetDict #import classes from the datasets module from huggingface
ds_train = Dataset.from_pandas(df_train).rename_column('score','label')
eval_ds= Dataset.from_pandas(eval_df)

In [ ]:
def tok_func(x): 
    return token(x["input"])
tok_func(ds_train[0])

### Import pre-trained language model DeBERTa**


In [ ]:
!pip install transformers
!pip install sentencepiece

In [ ]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer, T5ForConditionalGeneration #collect the pretrained model
model_nm = 'microsoft/deberta-v3-small'
tokenizer = AutoTokenizer.from_pretrained(model_nm) #create the tokenizer class from the pretrained model

In [ ]:
tok_ds = ds.map(lambda x: tokenizer(x["input"]), batched=True)
row = tok_ds[0]
row['input'], row['input_ids']

In [ ]:
tok_ds = tok_ds.rename_columns({'score':'labels'})

### Validation set

In [ ]:
anchors = df.anchor.unique()
np.random.seed(42)
np.random.shuffle(anchors)
anchors[:5]

In [ ]:
val_prop = 0.25
val_sz = int(len(anchors)*val_prop)
val_anchors = anchors[:val_sz]

In [ ]:
is_val = np.isin(df.anchor, val_anchors)
idxs = np.arange(len(df))
val_idxs = idxs[ is_val]
trn_idxs = idxs[~is_val]
len(val_idxs),len(trn_idxs)

In [ ]:
dds = DatasetDict({"train":tok_ds.select(trn_idxs),
             "test": tok_ds.select(val_idxs)})

## training model

In [ ]:
def corr(eval_pred): return {'pearson': np.corrcoef(*eval_pred)[0][1]}

In [ ]:
lr,bs = 8e-5,128
wd,epochs = 0.01,4

In [ ]:
def get_trainer(dds):
    args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
        evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
        num_train_epochs=epochs, weight_decay=wd, report_to='none')
    model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
    return Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                   tokenizer=tokz, compute_metrics=corr)

In [ ]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=wd, report_to='none')

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
               tokenizer=tokz, compute_metrics=corr)

In [ ]:
trainer.train();